In [2]:
from collections import namedtuple
from glob import glob
import json

import numpy as np
from PIL import Image
import random
import pandas as pd

from tqdm import tqdm

Label = namedtuple( "Label" , [

    "name"        , # The identifier of this label, e.g. "car", "person", ... .
                    # We use them to uniquely name a class

    "id"          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    "trainId"     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # "preparation" folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    "category"    , # The name of the category that this label belongs to

    "categoryId"  , # The ID of this category. Used to create ground truth images
                    # on category level.

    "hasInstances", # Whether this label distinguishes between single instances or not

    "ignoreInEval", # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    "color"       , # The color of this label
    ] )

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  "unlabeled"            ,  0 ,      255 , "void"            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  "ego vehicle"          ,  1 ,      255 , "void"            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  "rectification border" ,  2 ,      255 , "void"            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  "out of roi"           ,  3 ,      255 , "void"            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  "static"               ,  4 ,      255 , "void"            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  "dynamic"              ,  5 ,      255 , "void"            , 0       , False        , True         , (111, 74,  0) ),
    Label(  "ground"               ,  6 ,      255 , "void"            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  "road"                 ,  7 ,        0 , "flat"            , 1       , False        , False        , (128, 64,128) ),
    Label(  "sidewalk"             ,  8 ,        1 , "flat"            , 1       , False        , False        , (244, 35,232) ),
    Label(  "parking"              ,  9 ,      255 , "flat"            , 1       , False        , True         , (250,170,160) ),
    Label(  "rail track"           , 10 ,      255 , "flat"            , 1       , False        , True         , (230,150,140) ),
    Label(  "building"             , 11 ,        2 , "construction"    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  "wall"                 , 12 ,        3 , "construction"    , 2       , False        , False        , (102,102,156) ),
    Label(  "fence"                , 13 ,        4 , "construction"    , 2       , False        , False        , (190,153,153) ),
    Label(  "guard rail"           , 14 ,      255 , "construction"    , 2       , False        , True         , (180,165,180) ),
    Label(  "bridge"               , 15 ,      255 , "construction"    , 2       , False        , True         , (150,100,100) ),
    Label(  "tunnel"               , 16 ,      255 , "construction"    , 2       , False        , True         , (150,120, 90) ),
    Label(  "pole"                 , 17 ,        5 , "object"          , 3       , False        , False        , (153,153,153) ),
    Label(  "polegroup"            , 18 ,      255 , "object"          , 3       , False        , True         , (153,153,153) ),
    Label(  "traffic light"        , 19 ,        6 , "object"          , 3       , False        , False        , (250,170, 30) ),
    Label(  "traffic sign"         , 20 ,        7 , "object"          , 3       , False        , False        , (220,220,  0) ),
    Label(  "vegetation"           , 21 ,        8 , "nature"          , 4       , False        , False        , (107,142, 35) ),
    Label(  "terrain"              , 22 ,        9 , "nature"          , 4       , False        , False        , (152,251,152) ),
    Label(  "sky"                  , 23 ,       10 , "sky"             , 5       , False        , False        , ( 70,130,180) ),
    Label(  "person"               , 24 ,       11 , "human"           , 6       , True         , False        , (220, 20, 60) ),
    Label(  "rider"                , 25 ,       12 , "human"           , 6       , True         , False        , (255,  0,  0) ),
    Label(  "car"                  , 26 ,       13 , "vehicle"         , 7       , True         , False        , (  0,  0,142) ),
    Label(  "truck"                , 27 ,       14 , "vehicle"         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  "bus"                  , 28 ,       15 , "vehicle"         , 7       , True         , False        , (  0, 60,100) ),
    Label(  "caravan"              , 29 ,      255 , "vehicle"         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  "trailer"              , 30 ,      255 , "vehicle"         , 7       , True         , True         , (  0,  0,110) ),
    Label(  "train"                , 31 ,       16 , "vehicle"         , 7       , True         , False        , (  0, 80,100) ),
    Label(  "motorcycle"           , 32 ,       17 , "vehicle"         , 7       , True         , False        , (  0,  0,230) ),
    Label(  "bicycle"              , 33 ,       18 , "vehicle"         , 7       , True         , False        , (119, 11, 32) ),
    Label(  "license plate"        , -1 ,       -1 , "vehicle"         , 7       , False        , True         , (  0,  0,142) ),
]

In [16]:
instance_train_path = list(glob("data/gtFine/train/*/*labelIds.png"))
random.shuffle(instance_train_path)
instance_train_path = instance_train_path#[:10]
df = pd.DataFrame([{"Class": i, "Name": labels[i].name, "Occurences": 0, "Count": 0} for i in range(35)])
for path in tqdm(instance_train_path):
    img = Image.open(path)
    img = np.array(img)
    for i in range(35):
        count = int(np.sum(img == i))
        df.loc[i, "Count"] += count
        if count > 0:
            df.loc[i, "Occurences"] += 1
total = df["Count"].sum()
df["Percent"] = 100 * df["Count"] / total
df_sort = df.sort_values(by=["Percent"], ascending=False)
df_sort

  0%|          | 0/1877 [00:00<?, ?it/s]

100%|██████████| 1877/1877 [02:31<00:00, 12.37it/s]


,Class,Name,Occurences,Count,Percent
7,7,road,1848,1299082210,33.002167
11,11,building,1842,804335273,20.433508
21,21,vegetation,1826,603462734,15.330498
26,26,car,1801,257624472,6.544748
8,8,sidewalk,1762,211906099,5.383309
1,1,ego vehicle,1877,153039318,3.887844
23,23,sky,1702,146252589,3.715433
3,3,out of roi,1877,59377018,1.508427
4,4,static,1853,52345814,1.329804
6,6,ground,648,50847730,1.291747


: 

In [5]:
rgb = [
    (  0,  0,  0),
    (  0,  0,  0),
    (  0,  0,  0),
    (  0,  0,  0),
    (  0,  0,  0),
    (111, 74,  0),
    ( 81,  0, 81),
    (128, 64,128),
    (244, 35,232),
    (250,170,160),
    (230,150,140),
    ( 70, 70, 70),
    (102,102,156),
    (190,153,153),
    (180,165,180),
    (150,100,100),
    (150,120, 90),
    (153,153,153),
    (153,153,153),
    (250,170, 30),
    (220,220,  0),
    (107,142, 35),
    (152,251,152),
    ( 70,130,180),
    (220, 20, 60),
    (255,  0,  0),
    (  0,  0,142),
    (  0,  0, 70),
    (  0, 60,100),
    (  0,  0, 90),
    (  0,  0,110),
    (  0, 80,100),
    (  0,  0,230),
    (119, 11, 32),
    (  0,  0,142),
]

for i, c in enumerate(rgb):
    hex = "#" + "".join([format(i, '02x') for i in c])
    print(i, hex)

0 #000000
1 #000000
2 #000000
3 #000000
4 #000000
5 #6f4a00
6 #510051
7 #804080
8 #f423e8
9 #faaaa0
10 #e6968c
11 #464646
12 #66669c
13 #be9999
14 #b4a5b4
15 #966464
16 #96785a
17 #999999
18 #999999
19 #faaa1e
20 #dcdc00
21 #6b8e23
22 #98fb98
23 #4682b4
24 #dc143c
25 #ff0000
26 #00008e
27 #000046
28 #003c64
29 #00005a
30 #00006e
31 #005064
32 #0000e6
33 #770b20
34 #00008e
